In [ ]:
import geopandas as gpd
import pandas as pd

# Configuration
inputLocation = 'origin Network/'
scenarios = ['現況', '全線拆除', '部分拆除']
time_periods = ['晨峰', '昏峰']

# Load capacity dictionary
cap_dict = {}
with open('TRTS4S_Capacity.csv', 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip header
        parts = line.strip().split(',')
        class_num = int(parts[0])
        for i in range(1, len(parts)):
            lanes = i
            capacity = int(float(parts[i]))
            cap_dict[(class_num, lanes)] = capacity

def modify_network_base(input_file):
    """Apply base network modifications"""
    net = gpd.read_file(input_file)
    
    for index, row in net.iterrows():
        o, d = row['A'], row['B']
        
        # 市民大道 S0 調整
        if (o == 16642 and d == 6467):
            net.at[index, 'S0'] = 20

        # 台北橋民權西路車道數調整
        if (o == 6540 and d == 8087) or (o == 8086 and d == 6540):
            net.at[index, 'LANES_110'] = 2
        if (o == 8087 and d == 8109) or (o == 8109 and d == 9887) or (o == 9887 and d == 15501) or (o == 15501 and d == 8353):
            net.at[index, 'LANES_110'] = 3
        if (o == 8347 and d == 15500) or (o == 15500 and d == 9888) or (o == 9888 and d == 8110) or (o == 8110 and d == 10414) or (o == 10414 and d == 8086):
            net.at[index, 'LANES_110'] = 3

        # 台北橋機車道車道數調整
        if (o == 5202 and d == 8111) or (o == 8111 and d == 6542) or (o == 6544 and d == 5202):
            net.at[index, 'LANES_110'] = 3
        # 重陽橋機車道車道數調整
        if (o == 7534 and d == 6668) or (o == 6669 and d == 5831):
            net.at[index, 'LANES_110'] = 2

        # 百齡橋 class 調整
        # class 27 -> class 26
        # （省道橋梁分隔車道較寬 -> 車道較窄）
        if row['CLASS_110'] == 27:
            net.at[index, 'CLASS_110'] = 26
        
        # 基隆路 S0有誤
        # link 8892 <-> 15321
        if (o == 8892 and d == 15321) or (o == 15321 and d == 8892):
            net.at[index, 'S0'] = 20
        # 基隆路高架容量有誤
        if row['ID'] in [2620, 3314]:
            net.at[index, 'LANES_110'] = 2

        # 中山民權路口慢車道 S0更正
        if row['ID'] == 20400:
            net.at[index, 'S0'] = 20

        # 環河 洲美-重陽橋 S0更正
        if row['ID'] == 21942:
            net.at[index, 'S0'] = 20
        # 環河南向S0調整
        if row['ID'] in [4359, 8251, 3677]:
            net.at[index, 'S0'] = 60
        # 環河北向取消禁行機車
        # 改 class 為市區道路
        if row['ID'] in [4018,3646,10928,8271,8270,10923,8260,8252]:
            net.at[index, 'NMT_110'] = 0  # 取消禁行機車
            net.at[index, 'S0'] = 50
            net.at[index, 'CLASS_110'] = 69
        if row['ID'] in [8218,4354,4372,4370,8216,4342,478]:
            net.at[index, 'NMT_110'] = 0  # 取消禁行機車
            net.at[index, 'LANES_110'] += 1  # 增加一車道
            net.at[index, 'CLASS_110'] = 69
            net.at[index, 'S0'] = 50
        if row['ID'] in [8211,21941]:
            net.at[index, 'LANES_110'] += 1  # 增加一車道
            net.at[index, 'S0'] = 50

        # 國一圓山至內湖開放路肩
        # link 6325 -> 6326
        # link 6323 -> 6324
        # lanes = 3
        if (o == 6325 and d == 6326) or (o == 6323 and d == 6324):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 3
        
        # 國一內湖至汐止更正車道數
        #北向
        if (o == 6982 and d == 6882) or (o == 6882 and d == 6883) or (o == 6883 and d == 5032):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 3
        if (o == 5032 and d == 5033):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 5
        if (o == 5033 and d == 7193):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 3
        #南向
        if (o == 7192 and d == 5022):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 3
        if (o == 5022 and d == 5023):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 5
        if (o == 5023 and d == 6886) or (o == 6886 and d == 6887) or (o == 6887 and d == 6978):
            net.at[index, 'CLASS_110'] = 2
            net.at[index, 'LANES_110'] = 3
        
        # 修改容量
        class_num = int(row['CLASS_110'])
        lanes = int(row['LANES_110'])
        if (class_num, lanes) in cap_dict:
            net.at[index, 'capacity'] = cap_dict[(class_num, lanes)]
    
    return net

def add_ramp_connections(net):
    """Add bidirectional ramp connections"""
    new_rows_list = []
    
    for index, row in net.iterrows():
        # 新增洲美福國路北出南入匝道節點
        # ID = 40190, 40191 設為雙向
        if row['ID'] in [40190, 40191]:
            net.at[index, 'DIR'] = 2
            new_row = row.copy()
            new_row['A'] = row['B']
            new_row['B'] = row['A']
            new_row['ID'] = 40191
            new_rows_list.append(new_row)
    
    if new_rows_list:
        new_rows_gdf = gpd.GeoDataFrame(new_rows_list, crs=net.crs)
        net = pd.concat([net, new_rows_gdf], ignore_index=True)
    
    return net

def apply_reversible_lanes(gdf, time_period):
    """Apply reversible lane configurations"""
    if time_period == '晨峰':
        # 承德路7:00-9:00
        # [8299,6666]:(5,3) means (A,B)=(8299,6666), LANES_110=5; (A,B)=(6666,8299), LANES_110=3
        # [6666,6628,8293,15665,9978,8289]:(4,2)
        # [6631,6660,6642,8141,14039,8143,9883,6387]:(5,3)
        # 中山北路7:00-9:00
        # [6291,6696,6695,6629,8291]:(3,1)
        # 民族東路7:00-9:00
        # [8345,6682,9154,15473,6375,6683]:(3,1)
        # [6683,6373,6409,6398]:(4,2)
        # 復興北路 五常街至民權東路 07:00~09:00
        # [6753,8090]:(4,2)
        # 北安路 大直橋至大直街 07:00-09:00
        # [6812,16591]:(2,4)
        # 新生南路 辛亥路至和平東路 07:00~09:00
        # [5994,15176,6018]:(3,5)
        # 環河南路 開封街至忠孝西路 07:00~09:00
        # [8156,5303]:(3,1)
        # 重慶南路 中正橋頭至愛國西路 07:00~09:00
        # [9760,27354,9758,6085,6092,6091,6086]:(4,2)
        # [6086,6065]:(3,1)
        # [6065,6076]:(5,3)
        # [9060,15387,6093,8401,6076]:(2,4)
        # 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
        # [6056,15347]:(0,2)
        # 永福橋 永福橋頭至自來水南區營業處 07:00~09:00
        # [10016,9796]:(2,0)
        # [9796,27340,6104]:(3,1)
        # [6104,6102,6103]:(2,0)
        # 敦化南路 基隆路至仁愛路 07:00~09:00
        # [7736,7836,7833,5323,7834,7835]:(0,3)
        # [7835,6227,6224]:(1,3)
        # 南港路 南港橋頭至研究院路 07:00~09:00
        # [7037,15826,7025,7023,9259,16385,7045,10251,7046,16384,9270,7050]:(2,4)
        # 研究院路 南港水廠至忠孝東路 07:00~09:00
        # [7028,15831,8685,7029,15833]:(2,4)
        # 民權東路 國醫中心至成功路 07:00~09:00
        # [6904,15761,6874]:(2,4)
        # 中山北路酒泉街至民生東路
        # [6676,6392,9218,9219,9121,8108,6393,8107,8071,8079]:3
        # [8361,8357,8354,8351,8348,9220,9217,8344,6676]:1
        reversible_links_dict = {
            # 承德路7:00-9:00 format: (A,B): (forward_lanes, backward_lanes)
            (8299, 6666): (5, 3),
            # 承德路段落2
            (6666, 6628): (4, 2), (6628, 8293): (4, 2), (8293, 15665): (4, 2),
            (15665, 9978): (4, 2), (9978, 8289): (4, 2),
            # 承德路段落3
            (6631, 6660): (5, 3), (6660, 6642): (5, 3), (6642, 8141): (5, 3),
            (8141, 14039): (5, 3), (14039, 8143): (5, 3), (8143, 9883): (5, 3),
            (9883, 6387): (5, 3),
            # 中山北路7:00-9:00
            (6291, 6696): (3, 1), (6696, 6695): (3, 1), (6695, 6629): (3, 1),
            (6629, 8291): (3, 1),
            # 民族東路7:00-9:00 段落1
            (8345, 6682): (3, 1), (6682, 9154): (3, 1), (9154, 15473): (3, 1),
            (15473, 6375): (3, 1), (6375, 6683): (3, 1),
            # 民族東路7:00-9:00 段落2
            (6683, 6373): (4, 2), (6373, 6409): (4, 2), (6409, 6398): (4, 2),
            # 復興北路 五常街至民權東路 07:00~09:00
            (6753, 8090): (4, 2),
            # 北安路 大直橋至大直街 07:00-09:00
            (6812, 16591): (2, 4),
            # 新生南路 辛亥路至和平東路 07:00~09:00
            (5994, 15176): (3, 5), (15176, 6018): (3, 5),
            # 環河南路 開封街至忠孝西路 07:00~09:00
            (8156, 5303): (3, 1),
            # 重慶南路 中正橋頭至愛國西路 07:00~09:00 段落1
            (9760, 27354): (4, 2), (27354, 9758): (4, 2), (9758, 6085): (4, 2),
            (6085, 6092): (4, 2), (6092, 6091): (4, 2), (6091, 6086): (4, 2),
            # 重慶南路段落2
            (6086, 6065): (3, 1),
            # 重慶南路段落3
            (6065, 6076): (5, 3),
            # 重慶南路段落4
            (9060, 15387): (2, 4), (15387, 6093): (2, 4), (6093, 8401): (2, 4),
            (8401, 6076): (2, 4),
            # 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
            (6056, 15347): (0, 2),
            # 永福橋 永福橋頭至自來水南區營業處 07:00~09:00 段落1
            (10016, 9796): (2, 0),
            # 永福橋段落2
            (9796, 27340): (3, 1), (27340, 6104): (3, 1),
            # 永福橋段落3
            (6104, 6102): (2, 0), (6102, 6103): (2, 0),
            # 敦化南路 基隆路至仁愛路 07:00~09:00 段落1
            (7736, 7836): (0, 3), (7836, 7833): (0, 3), (7833, 5323): (0, 3),
            (5323, 7834): (0, 3), (7834, 7835): (0, 3),
            # 敦化南路段落2
            (7835, 6227): (1, 3), (6227, 6224): (1, 3),
            # 南港路 南港橋頭至研究院路 07:00~09:00
            (7037, 15826): (2, 4), (15826, 7025): (2, 4), (7025, 7023): (2, 4),
            (7023, 9259): (2, 4), (9259, 16385): (2, 4), (16385, 7045): (2, 4),
            (7045, 10251): (2, 4), (10251, 7046): (2, 4), (7046, 16384): (2, 4),
            (16384, 9270): (2, 4), (9270, 7050): (2, 4),
            # 研究院路 南港水廠至忠孝東路 07:00~09:00
            (7028, 15831): (2, 4), (15831, 8685): (2, 4), (8685, 7029): (2, 4),
            (7029, 15833): (2, 4),
            # 民權東路 國醫中心至成功路 07:00~09:00
            (6904, 15761): (2, 4), (15761, 6874): (2, 4),
            # 中山北路酒泉街至民生東路 段落1
            (6676, 6392): (3, 1), (6392, 9218): (3, 1), (9218, 9219): (3, 1),
            (9219, 9121): (3, 1), (9121, 8108): (3, 1), (8108, 6393): (3, 1),
            (6393, 8107): (3, 1), (8107, 8071): (3, 1), (8071, 8079): (3, 1),
            # 中山北路酒泉街至民生東路 段落2 (反向)
            (8361, 8357): (1, 3), (8357, 8354): (1, 3), (8354, 8351): (1, 3),
            (8351, 8348): (1, 3), (8348, 9220): (1, 3), (9220, 9217): (1, 3),
            (9217, 8344): (1, 3), (8344, 6676): (1, 3)
        }
    else:  # 昏峰
        # 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
        # [6056,15347]:(0,2)
        # 舊宗路 行善路至南京東路6段 17:30~19:30
        # [6962,15776,7986]:(4,2)
        # 瑞湖街 舊宗路2段171巷至民權東路6段11巷 17:00~19:00
        # [7013,15768,7008,6910]:(2,0)
        # 民生西路 迪化街至環河北路 16:00~19:00
        # [6548,9106,6551,8057]:(1,3)
        # 基隆路 臺北市調處至基隆高架道路樂業匝道 17:00~19:30
        # [6225,6228,6205,15323,6194,15321]:(2,4)
        # [6201,6225]:swap A B
        reversible_links_dict = {
            # 水源路 水源路35巷至師大路口 07:00~09:00 17:00~19:00
            (6056, 15347): (0, 2),
            # 舊宗路 行善路至南京東路6段 17:30~19:30
            (6962, 15776): (4, 2), (15776, 7986): (4, 2),
            # 瑞湖街 舊宗路2段171巷至民權東路6段11巷 17:00~19:00
            (7013, 15768): (2, 0), (15768, 7008): (2, 0), (7008, 6910): (2, 0),
            # 民生西路 迪化街至環河北路 16:00~19:00
            (6548, 9106): (1, 3), (9106, 6551): (1, 3), (6551, 8057): (1, 3),
            # 基隆路 臺北市調處至基隆高架道路樂業匝道 17:00~19:30
            (6225, 6228): (2, 4), (6228, 6205): (2, 4), (6205, 15323): (2, 4),
            (15323, 6194): (2, 4), (6194, 15321): (2, 4)
        }
    
    # Apply reversible lane configurations
    for index, row in gdf.iterrows():
        A, B = row['A'], row['B']
        lanes = row['LANES_110']
        class_num = row['CLASS_110']
        
        # 檢查是否為可調撥車道
        if (A, B) in reversible_links_dict:
            lanes = reversible_links_dict[(A, B)][0]  # 取得前進方向車道數
        elif (B, A) in reversible_links_dict:
            lanes = reversible_links_dict[(B, A)][1]  # 取得反向車道數
        elif time_period == '昏峰' and (A, B) == (6201, 6225):
            # 特例處理，A和B互換，車道數不變
            gdf.at[index, 'A'] = B
            gdf.at[index, 'B'] = A
        
        # Calculate capacity based on updated lanes and class
        capacity = cap_dict.get((class_num, lanes), 0)
        
        # Update lanes and capacity
        gdf.at[index, 'LANES_110'] = lanes
        gdf.at[index, 'capacity'] = capacity
    
    return gdf

def apply_scenario_specific_changes(gdf, scenario):
    """Apply scenario-specific modifications"""
    for index, row in gdf.iterrows():
        A, B = row['A'], row['B']
        lanes = row['LANES_110']
        class_num = row['CLASS_110']
        
        # 新生北路平面 車道數有誤（僅部分拆除路網）
        # link 6597 -> 6598, 8414 -> 9214
        # lanes = 2
        if scenario == '部分拆除':
            if (A == 6597 and B == 6598) or (A == 8414 and B == 9214):
                lanes = 2
                gdf.at[index, 'LANES_110'] = lanes
                gdf.at[index, 'capacity'] = cap_dict.get((class_num, lanes), 0)
            #新生高民族東路上橋端容量調整
            if (A == 6681 and B == 15472) or (A == 15472 and B == 6683) or (A == 6373 and B == 6684):
                class_num = 13
                gdf.at[index, 'capacity'] = cap_dict.get((class_num, lanes), 0)
        
        # 中山北路四段 車道數調整（僅全線拆除路網）
        # link 8342 -> 6675
        # link 6675 -> 8343
        # lanes = 4
        elif scenario == '全線拆除':
            if (A == 8342 and B == 6675) or (A == 6675 and B == 8343):
                lanes = 4
                gdf.at[index, 'LANES_110'] = lanes
                gdf.at[index, 'capacity'] = cap_dict.get((class_num, lanes), 0)
    
    return gdf

def convert_to_dat_from_gdf(gdf, output_file, time_period):
    """Convert GeoDataFrame to DAT format"""
    rows = []
    
    for index, row in gdf.iterrows():
        if row["LEVEL_110"] in [1, 2, 3, 4, 5, 6, 7, 11]:
            origin = row["A"]
            dest = row["B"]
            capacity = cap_dict.get((row["CLASS_110"], row["LANES_110"]), 0)
            
            if row["CLASS_110"] == 99:
                capacity = 9999
            #4359,8218,8251,8252,3677,8260,10923,8270,8271,10928,3646,4018 ID, half capacity
            if row["ID"] in [4359, 8218, 8251, 8252, 3677, 8260, 10923, 8270, 8271, 10928, 3646, 4018]:
                capacity = capacity / 2
            #2343 ID, 0.85 capacity
            if row["ID"] == 2343:
                capacity = capacity * 0.85
            
            length = float(row["LENGTH"]) or 1000
            try:
                fft = length / float(row["S0"]) * 60
            except:
                # if ID is not 40438, then give S0=32
                fft = length / 32 * 60 if row["ID"] != 40438 else 1000
            
            alpha = row["ALPHA"] or 0
            beta = row["BETA"] or 0
            speedLimit = 0
            level = int(row["LEVEL_110"]) or 0
            NMoto = row["NMT_110"] or 0
            if row["CLASS_110"] == 84:
                NCar = 1
            else:
                NCar = 0
            
            if capacity == 0:
                continue
            
            rows.append({
                'origin': origin, 'dest': dest, 'capacity': capacity,
                'length': length, 'fft': fft, 'alpha': alpha, 'beta': beta,
                'speedLimit': speedLimit, 'LinkType': level, 'NMoto': NMoto, 'NCar': NCar
            })
    
    # 承德福國路口節點新增 (藉區心線連結兩個node)
    rows.extend([
        {'origin': '15730', 'dest': '27656', 'capacity': '9999.0', 'length': '0.1', 
            'fft': '1.0', 'alpha': '0.0', 'beta': '0.0', 'speedLimit': '0.0', 
            'LinkType': '0', 'NMoto': '0', 'NCar': '0'},
        {'origin': '27656', 'dest': '15730', 'capacity': '9999.0', 'length': '0.1', 
            'fft': '1.0', 'alpha': '0.0', 'beta': '0.0', 'speedLimit': '0.0', 
            'LinkType': '0', 'NMoto': '0', 'NCar': '0'}
    ])
    
    # Add supplement data
    df_supplement = pd.read_csv('network_supplement.dat', sep='\t', header=None, 
                                names=['origin', 'dest', 'capacity', 'length', 'fft', 'alpha', 'beta', 'speedLimit', 'LinkType', 'NMoto', 'NCar'])
    df_supplement['LinkType'] = 0
    df_supplement['NMoto'] = 0
    df_supplement['NCar'] = 0
    
    df_out = pd.concat([pd.DataFrame(rows), df_supplement], ignore_index=True)
    df_out.to_csv(output_file, index=False, sep='\t')

# Main processing loop
for scenario in scenarios:
    print(f"Processing scenario: {scenario}")
    
    # Load and modify base network
    input_file = f"{inputLocation}Net_modified_{scenario}.shp"
    net = modify_network_base(input_file)
    net = add_ramp_connections(net)
    
    for time_period in time_periods:
        print(f"  Processing {time_period}")
        
        # Apply time-specific modifications
        gdf = apply_reversible_lanes(net.copy(), time_period)
        gdf = apply_scenario_specific_changes(gdf, scenario)
        
        # Save files
        output_prefix = f"Net_modified_{scenario}_{time_period}調撥"
        shp_file = f"{output_prefix}.shp"
        # csv_file = f"{output_prefix}.csv"
        dat_file = f"{output_prefix}.dat"
        
        # Save shapefile and CSV
        gdf.to_file(shp_file, driver='ESRI Shapefile')
        # gdf.to_csv(csv_file, index=False)
        
        # Convert to DAT format using the geodataframe directly
        convert_to_dat_from_gdf(gdf, dat_file, time_period)

print("All network modifications completed successfully.")